In [1]:
import pandas as pd
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

In [2]:
bert_df = pd.read_csv('BM25.csv')

In [3]:
bert_df

,resume_text,company_name,title,description,salary,location,formatted_experience_level,skills_desc,work_type,zip_code,job_matching,bm25_score
0,"DIRECTOR, GLOBAL BUSINESS DEVELOPMENT...",MaxLinear,"Senior Manager, Wireless Infrastructure",Responsibilities\n\nMaxLinear is seeking a Sen...,195040.000000,"Carlsbad, CA",Mid-Senior level,NaN,FULL_TIME,92008.0,"Senior Manager, Wireless Infrastructure Respon...",859.751124
1,"DIRECTOR, GLOBAL BUSINESS DEVELOPMENT...",SolarEdge Technologies,Commercial Sales Manager (TX),"SolarEdge (NASDAQ: SEDG), is a global leader i...",130000.000000,"Austin, TX",Mid-Senior level,NaN,FULL_TIME,78701.0,Commercial Sales Manager (TX) SolarEdge (NASDA...,830.282005
2,"DIRECTOR, GLOBAL BUSINESS DEVELOPMENT...",CornerStone Title Company,Sales Representative,Job Description\n\nReal Life. Real Relationshi...,70000.000000,"Roseville, CA",Entry level,NaN,FULL_TIME,95661.0,Sales Representative Job Description\n\nReal L...,817.012934
3,"DIRECTOR, GLOBAL BUSINESS DEVELOPMENT...",Kenvue,"Director, North America Business Product Owner...",Job Description\n\nKenvue is currently recruit...,235750.000000,"Skillman, NJ",Director,NaN,FULL_TIME,8558.0,"Director, North America Business Product Owner...",811.839805
4,"DIRECTOR, GLOBAL BUSINESS DEVELOPMENT...",Amazon Web Services (AWS),"Partner Sales Manager, Startups, AGS - Startup...",Description\n\nAs a Partner Sales Manager for ...,177800.000000,"San Francisco, CA",Mid-Senior level,NaN,FULL_TIME,94101.0,"Partner Sales Manager, Startups, AGS - Startup...",808.299916
...,...,...,...,...,...,...,...,...,...,...,...,...
195,ENGLISH LANGUAGE ARTS TEACHER S...,Douglas County School District,Teacher - Mathematics - Middle,Please complete this application using your fu...,116582.000000,"Highlands Ranch, CO",Entry level,NaN,FULL_TIME,NaN,Teacher - Mathematics - Middle Please complete...,578.533221
196,ENGLISH LANGUAGE ARTS TEACHER S...,Uncommon Schools,"K-12 Teacher - Elementary, Middle, High School...",Company Description\n\nUncommon Schools is a n...,64700.000000,"Rochester, NY",Associate,NaN,FULL_TIME,14604.0,"K-12 Teacher - Elementary, Middle, High School...",572.546873
197,ENGLISH LANGUAGE ARTS TEACHER S...,Mastery Schools,Mastery Teaching Residency,"About Mastery:\n\nFounded in 2001, Mastery Sch...",286497.116458,"Camden, NJ",Entry level,NaN,FULL_TIME,8102.0,Mastery Teaching Residency About Mastery:\n\nF...,571.186284
198,ENGLISH LANGUAGE ARTS TEACHER S...,Western Governors University,Instructor - Nutrition (Part-time),The salary range for this role takes into acco...,89897.600000,United States,Mid-Senior level,NaN,PART_TIME,NaN,Instructor - Nutrition (Part-time) The salary ...,565.394347


In [4]:
if 'eval_score' not in bert_df.columns:
    bert_df['eval_score'] = None

In [6]:
resume_index = 0
job_index = 0
total_jobs = len(bert_df)
jobs_per_resume = 10
total_resumes = total_jobs // jobs_per_resume

progress_bar = widgets.IntProgress(value=0, min=0, max=total_jobs, description='Progress:', bar_style='info')

def get_df_idx():
    return resume_index * jobs_per_resume + job_index

def render():
    clear_output()

    if resume_index >= total_resumes:
        progress_bar.value = total_jobs
        display(progress_bar)
        display(HTML("<h3 style='color: green;'>✅ Evaluation complete. CSV saved as <code>manual_evaluation_done.csv</code>.</h3>"))
        return

    df_idx = get_df_idx()

    resume_text = bert_df.loc[df_idx, 'resume_text']
    job_text = bert_df.loc[df_idx, 'job_matching']
    bm25_score = bert_df.loc[df_idx, 'bm25_score']

    progress_bar.value = df_idx
    display(progress_bar)

    display(HTML(f"""
    <div style="white-space: pre-wrap; font-size: 18px; border: 2px solid #ccc; padding: 10px; margin-top: 10px; margin-bottom: 10px;">
        <b>Resume {resume_index + 1}:</b><br><br>{resume_text}
    </div>
    """))

    display(HTML(f"""
    <div style="white-space: pre-wrap; font-size: 16px; border: 1px solid #aaa; padding: 8px; margin-bottom: 10px;">
        <b>Job Match {job_index + 1} of 10 (BM25 Score: {bm25_score:.2f}):</b><br><br>{job_text}
    </div>
    """))

    relevant_button.disabled = False
    not_relevant_button.disabled = False
    display(widgets.HBox([relevant_button, not_relevant_button]))

def save_eval_and_advance(score):
    global job_index, resume_index

    df_idx = get_df_idx()
    bert_df.at[df_idx, 'eval_score'] = score

    job_index += 1
    if job_index >= jobs_per_resume:
        job_index = 0
        resume_index += 1

    if resume_index >= total_resumes:
        bert_df.to_csv("manual_evaluation_done.csv", index=False)

    render()

def on_relevant_clicked(b):
    relevant_button.disabled = True
    not_relevant_button.disabled = True
    save_eval_and_advance(1)

def on_not_relevant_clicked(b):
    relevant_button.disabled = True
    not_relevant_button.disabled = True
    save_eval_and_advance(0)

relevant_button = widgets.Button(description="👍 Relevant", button_style='success')
not_relevant_button = widgets.Button(description="👎 Not Relevant", button_style='danger')

relevant_button.on_click(on_relevant_clicked)
not_relevant_button.on_click(on_not_relevant_clicked)

render()


IntProgress(value=0, bar_style='info', description='Progress:', max=200)